In [1]:
!pip install dash plotly

In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.colors as colors
from datetime import datetime

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("SpaceX_Data_Cleaned.csv")

# Convert 'Date' column to datetime format if necessary
spacex_df['Date'] = pd.to_datetime(spacex_df['Date'])

max_payload = spacex_df['PayloadMass'].max()
min_payload = spacex_df['PayloadMass'].min()
max_date = spacex_df['Date'].max()
min_date = spacex_df['Date'].min()

# Extract the years from the date range
min_year = min_date.year
max_year = max_date.year

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Dashboard',
                                        style={'textAlign': 'center', 'color': 'Blue',
                                               'font-size': 40}),
                                # Dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'All Sites'},
                                                 {'label': 'CCSFS SLC 40', 'value': 'CCSFS SLC 40'},
                                                 {'label': 'VAFB SLC 4E', 'value': 'VAFB SLC 4E'},
                                                 {'label': 'KSC LC 39A', 'value': 'KSC LC 39A'},
                                             ],
                                             placeholder='Select a Launch Site Here',
                                             value='All Sites',
                                             searchable=True),
                                html.Br(),

                                # Pie chart for successful launches
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Slider for payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks={i: '{}'.format(i) for i in range(0, 10001, 1000)},
                                                value=[min_payload, max_payload]),

                                html.P("Date range (Years):"),
                                # Slider for year range
                                dcc.RangeSlider(id='date-slider',
                                                min=min_year,
                                                max=max_year,
                                                step=1,
                                                marks={year: str(year) for year in range(min_year, max_year + 1, 1)},
                                                value=[min_year, max_year]),

                                # Scatter chart for payload and success correlation
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                html.Br(),
                                 dcc.Dropdown(id='orbit-dropdown',
                                             options=[
                                                 {'label': 'All Oribits', 'value': 'All Oribits'},
                                                 {'label': 'VLEO', 'value': 'VLEO'},
                                                 {'label': 'ISS', 'value': 'ISS'},
                                                 {'label': 'GTO', 'value': 'GTO'},
                                                 {'label': 'LEO', 'value': 'LEO'},
                                                 {'label': 'PO', 'value': 'PO'},
                                                 {'label': 'SSO', 'value': 'SSO'},
                                                 {'label': 'MEO', 'value': 'MEO'},
                                                 {'label': 'TLI', 'value': 'TLI'},
                                                 {'label': 'ES-L1', 'value': 'ES-L1'},
                                                 {'label': 'HEO', 'value': 'HEO'},
                                                 {'label': 'SO', 'value': 'SO'},
                                             ],
                                             placeholder='Select a Orbit Here',
                                             value='All Oribits',
                                             searchable=True),
                                html.Br(),
                                # Pie chart for successful launches
                                html.Div(dcc.Graph(id='orbit-pie-chart')),
                                ])


@app.callback(Output(component_id='orbit-pie-chart', component_property='figure'),
              Input(component_id='orbit-dropdown', component_property='value'))
def get_pie_chart(orbit):
    if orbit == 'All Oribits':
        fig = px.pie(spacex_df.groupby('Orbit')['Class'].mean(),
                     names=spacex_df.groupby('Orbit')['Orbit'].first(),
                     title='Total Success Launches by Orbit')
    else:
        fig = px.pie(values=spacex_df[spacex_df['Orbit'] == str(orbit)]['Class'].value_counts(normalize=True),
                     names=spacex_df['Class'].unique(),
                     title='Total Success Launches for Orbit {}'.format(orbit))

    # Set colors to shades of blue and gray
    fig.update_traces(marker=dict(colors=['#1f77b4', '#7f7f7f']), textinfo='label+percent')
    return fig


# Callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(launch_site):
    if launch_site == 'All Sites':
        fig = px.pie(spacex_df.groupby('LaunchSite')['Class'].mean(),
                     names=spacex_df.groupby('LaunchSite')['LaunchSite'].first(),
                     title='Total Success Launches by Site')
    else:
        fig = px.pie(values=spacex_df[spacex_df['LaunchSite'] == str(launch_site)]['Class'].value_counts(normalize=True),
                     names=spacex_df['Class'].unique(),
                     title='Total Success Launches for Site {}'.format(launch_site))

    # Set colors to shades of blue and gray
    fig.update_traces(marker=dict(colors=['#1f77b4', '#7f7f7f']), textinfo='label+percent')
    return fig

# Callback function for `site-dropdown`, `payload-slider`, and `date-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value'),
               Input(component_id='date-slider', component_property='value')])
def get_payload_chart(launch_site, payload_mass, date_range):
    # Filter data by the selected date range (years)
    min_year_selected, max_year_selected = date_range
    filtered_df = spacex_df[(spacex_df['Date'].dt.year >= min_year_selected) &
                            (spacex_df['Date'].dt.year <= max_year_selected) &
                            spacex_df['PayloadMass'].between(payload_mass[0], payload_mass[1])]

    if launch_site == 'All Sites':
        fig = px.scatter(filtered_df,
                         x="PayloadMass",
                         y="Class",
                         hover_data=['LaunchSite','Date'],
                         title=f'Correlation Between Payload and Success for All Sites ({min_year_selected}-{max_year_selected})')
    else:
        df = filtered_df[filtered_df['LaunchSite'] == str(launch_site)]
        fig = px.scatter(df,
                         x="PayloadMass",
                         y="Class",
                         hover_data=['LaunchSite','Date'],
                         title=f'Correlation Between Payload and Success for Site {launch_site} ({min_year_selected}-{max_year_selected})')

    # Update color to shades of blue and gray
    fig.update_traces(marker=dict(color=filtered_df['Class'].map({1: '#1f77b4', 0: '#7f7f7f'})))
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()


C:\Users\Omar\AppData\Local\Temp\ipykernel_20796\2773190694.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Omar\AppData\Local\Temp\ipykernel_20796\2773190694.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
